In [ ]:
#I am using this notebook as practice to follow along with evolving hockeys tutorial
#on how they calculate their expected goal value. link to the tutorial is here
#https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [1]:
import pandas as pd
import numpy as np

In [27]:
#df_import = pd.read_csv('data/washington_sample.csv')
df = pd.read_csv('data/march_4th_sample.csv')

In [28]:
#This page was helpful https://stackoverflow.com/questions/39109045/numpy-where-with-multiple-conditions
#Conditions for extracting the zone
col = 'event_description'
conditions = [df[col].str.contains('Off. Zone', na=False), df[col].str.contains('Def. Zone', na=False), 
              df[col].str.contains('Neu. Zone', na=False)]
choices = ['Off', 'Def','Neu']
fenwick = ['SHOT', 'MISS', 'GOAL']
corsi = ['SHOT', 'MISS', 'GOAL', 'BLOCK']
correction = ['Tip-In', 'Wrap-around', 'Deflected']
even_strength = ['5v5','4v4','3v3']
uneven_strength = ['5v4','4v5','5v3','3v5','4v3','3v4','5vE','Ev5','4vE','Ev4','3vE','Ev3']

In [29]:
df['event_distance'] = ((89 - abs(df['coords_x']))**2 + df['coords_y']**2)**(1/2)
df['event_angle'] = abs(np.arctan(df['coords_y'] / (89 - abs(df['coords_x']))) * (180 / np.pi))
df['pre_event_zone'] = np.select(conditions, choices, default=np.nan)
df['pre_event_zone_2'] = np.where((df.pre_event_zone == 'Def') & 
                                  (df.event_type == 'BLOCK'), 
                                  'Off',df.pre_event_zone)
df['pre_event_zone_3'] = np.where((df.event_type.isin(fenwick)) & 
                                  (df.event_zone == 'Def') & 
                                  (df.pbp_distance <= 64),
                                  'Off',df.pre_event_zone)
df['event_distance_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((abs(df.coords_x)+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_distance_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((df.coords_x+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_angle_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(abs(df['coords_x'])+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_angle_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(df['coords_x']+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_zone_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.event_zone == 'Def') &
                                     (df.pbp_distance <= 64),
                                     'Off',
                                     df.event_zone)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     'Ev1',
                                     df.game_strength_state)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     '1vE',
                                     df.game_strength_state)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     1,
                                     df.home_skaters)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     0,
                                     df.home_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     0,
                                     df.away_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     1,
                                     df.away_skaters)

In [30]:
event_interest = ["FAC", "GOAL", "BLOCK", "SHOT", "MISS", "HIT", "TAKE", "GIVE"]
filter_1 = df.event_type.isin(event_interest)
filter_2 = df.game_period < 5
filter_3 = ~df.event_description.str.contains('penalty shot', na=False)
#filter_4 = df.coords_x.isna()
#filter_5 = df.coords_y.isna()
filter_4 = ~((df.coords_x == 0) &
             (df.coords_y == 0) &
             (df.event_type.isin(corsi)) &
             ((df.pbp_distance != 90) & (df.event_type.isin(fenwick))))

In [31]:
#df_2 = df[filter_1 & filter_2 & filter_3 & filter_4 & filter_5 & filter_6].copy()
df = df[filter_1 & filter_2 & filter_3 & filter_4].copy()

In [32]:
df.dropna(subset=['coords_x','coords_y'],inplace=True)

In [33]:
#df = df.groupby(['season', 'game_id', 'game_period']).apply(lambda x: x.sort_values(['event_index']))
df_3 = df.sort_values(['season', 'game_id', 'game_period','event_index'],inplace=True)

In [35]:
df['seconds_since_last'] = df.game_seconds - df.game_seconds.shift(1)
df['event_type_last'] = df.event_type.shift(1)
df['event_team_last'] = df.event_team.shift(1)
df['event_strength_last'] = df.game_strength_state.shift(1)
df['coords_x_last'] = df.coords_x.shift(1)
df['coords_y_last'] = df.coords_y.shift(1)

In [10]:
filter_5 = df.event_type.isin(fenwick)
filter_6 = df.game_strength_state.isin(even_strength)

In [12]:
df = df[filter_5 & filter_6].copy()
df.dropna(subset=['coords_x','coords_y'],inplace=True)

<ipython-input-12-f1876920925e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter_5 & filter_6].copy()


In [15]:
df['same_team_last']     = np.where(df.event_team == df.event_team_last,1,0)
df['is_home']            = np.where(df.event_team == df.home_team,1,0)
df['score_state']        = np.where(df.is_home == 1, 
                                    df.home_score - df.away_score, 
                                    df.away_score - df.home_score)
df['event_detail']       = np.where(df.event_detail.isna(), 'Wrist', df.event_detail)
df['distance_from_last'] = (((df.coords_x - df.coords_x_last)**2) + 
                            ((df.coords_y - df.coords_y_last)**2))**(1/2)

In [17]:
df.rename(columns={"event_distance": "shot_distance", "event_angle": "shot_angle"}, inplace=True)

In [24]:
df.loc[:,~df.columns.duplicated()]

season     game_id   game_date  \
season   game_id    game_period                                          
20202021 2020020163 1           12    20202021  2020020163  2021-03-04   
                                16    20202021  2020020163  2021-03-04   
                                17    20202021  2020020163  2021-03-04   
                                28    20202021  2020020163  2021-03-04   
                                30    20202021  2020020163  2021-03-04   
...                                        ...         ...         ...   
         2020020742 3           9927  20202021  2020020742  2021-03-03   
                                9931  20202021  2020020742  2021-03-03   
                                9932  20202021  2020020742  2021-03-03   
                                9946  20202021  2020020742  2021-03-03   
                                9950  20202021  2020020742  2021-03-03   

                                     session  event_index  game_period  \
season   game_id    game_period                                          
20202021 2020020163 1           12         R           13            1   
                                16         R           17            1   
                                17         R           18            1   
                                28         R           29            1   
                                30         R           31            1   
...                                      ...          ...          ...   
         2020020742 3           9927       R          469            3   
                                9931       R          473            3   
                                9932       R          474            3   
                                9946       R          488            3   
                                9950       R          492            3   

                                      game_seconds clock_time event_type  \
season   game_id    game_period                                            
20202021 2020020163 1           12              78      18:42       SHOT   
                                16             114      18:06       MISS   
                                17             137      17:43       SHOT   
                                28             197      16:43       MISS   
                                30             204      16:36       SHOT   
...                                            ...        ...        ...   
         2020020742 3           9927          3191       6:49       SHOT   
                                9931          3219       6:21       SHOT   
                                9932          3227       6:13       GOAL   
                                9946          3267       5:33       SHOT   
                                9950          3287       5:13       SHOT   

                                                                      event_description  \
season   game_id    game_period                                                           
20202021 2020020163 1           12    NYI ONGOAL - #12 BAILEY, Wrist, Off. Zone, 45 ft.   
                                16    BUF #78 BRYSON, Wrist, Wide of Net, Off. Zone,...   
                                17    BUF ONGOAL - #78 BRYSON, Wrist, Off. Zone, 57 ft.   
                                28    NYI #2 LEDDY, Wrist, Wide of Net, Def. Zone, 1...   
                                30      NYI ONGOAL - #2 LEDDY, Wrist, Off. Zone, 23 ft.   
...                                                                                 ...   
         2020020742 3           9927  ARI ONGOAL - #83 GARLAND, Wrist, Off. Zone, 6 ft.   
                                9931  L.A ONGOAL - #77 CARTER, Tip-In, Off. Zone, 19...   
                                9932  L.A #13 VILARDI(6), Tip-In, Off. Zone, 11 ft.A...   
                                9946  L.A ONGOAL - #26 WALKER, Wrist, Off. Zone, 50 ft.   
                                9

In [34]:
df_3.head()

,season,game_id,game_date,session,event_index,game_period,game_seconds,clock_time,event_type,event_description,...,shift_index,pred_goal,pre_event_zone,pre_event_zone_2,pre_event_zone_3,event_distance_long,event_distance_long_2,event_angle_long,event_angle_long_2,event_zone_long
6,20202021,2020020163,2021-03-04,R,7,1,0,20:00,FAC,BUF won Neu. Zone - BUF #27 LAZAR vs NYI #53 C...,...,2,NaN,Neu,Neu,Neu,89.000000,89.000000,0.000000,0.000000,Neu
7,20202021,2020020163,2021-03-04,R,8,1,43,19:17,BLOCK,"NYI #2 LEDDY BLOCKED BY BUF #27 LAZAR, Wrist,...",...,2,NaN,Def,Off,Def,21.189620,21.189620,19.290046,19.290046,Off
11,20202021,2020020163,2021-03-04,R,12,1,67,18:53,HIT,"BUF #33 MILLER HIT NYI #29 NELSON, Def. Zone",...,5,NaN,Def,Def,Def,7.071068,7.071068,8.130102,8.130102,Def
12,20202021,2020020163,2021-03-04,R,13,1,78,18:42,SHOT,"NYI ONGOAL - #12 BAILEY, Wrist, Off. Zone, 45 ft.",...,5,0.018312,Off,Off,Off,45.541190,45.541190,8.841815,8.841815,Off
15,20202021,2020020163,2021-03-04,R,16,1,78,18:42,FAC,BUF won Def. Zone - BUF #9 EICHEL vs NYI #29 N...,...,6,NaN,Def,Def,Def,29.732137,29.732137,47.726311,47.726311,Def


In [25]:
df_2 = df['game_id'].reset_index()

ValueError: cannot insert game_id, already exists

In [23]:
df.head()

season     game_id   game_date session  \
season   game_id    game_period                                                
20202021 2020020163 1           12  20202021  2020020163  2021-03-04       R   
                                16  20202021  2020020163  2021-03-04       R   
                                17  20202021  2020020163  2021-03-04       R   
                                28  20202021  2020020163  2021-03-04       R   
                                30  20202021  2020020163  2021-03-04       R   

                                    event_index  game_period  game_seconds  \
season   game_id    game_period                                              
20202021 2020020163 1           12           13            1            78   
                                16           17            1           114   
                                17           18            1           137   
                                28           29            1           197   
                                30           31            1           204   

                                   clock_time event_type  \
season   game_id    game_period                            
20202021 2020020163 1           12      18:42       SHOT   
                                16      18:06       MISS   
                                17      17:43       SHOT   
                                28      16:43       MISS   
                                30      16:36       SHOT   

                                                                    event_description  \
season   game_id    game_period                                                         
20202021 2020020163 1           12  NYI ONGOAL - #12 BAILEY, Wrist, Off. Zone, 45 ft.   
                                16  BUF #78 BRYSON, Wrist, Wide of Net, Off. Zone,...   
                                17  BUF ONGOAL - #78 BRYSON, Wrist, Off. Zone, 57 ft.   
                                28  NYI #2 LEDDY, Wrist, Wide of Net, Def. Zone, 1...   
                                30    NYI ONGOAL - #2 LEDDY, Wrist, Off. Zone, 23 ft.   

                                    ... seconds_since_last event_type_last  \
season   game_id    game_period     ...                                      
20202021 2020020163 1           12  ...               11.0             HIT   
                                16  ...               36.0             FAC   
                                17  ...               23.0            MISS   
                                28  ...               36.0             HIT   
                                30  ...                7.0             HIT   

                                   event_team_last event_strength_last  \
season   game_id    game_period                                          
20202021 2020020163 1           12             BUF                 5v5   
                                16             BUF                 5v5   
                                17             BUF                 5v5   
                                28             BUF                 5v5   
                                30             NYI                 5v5   

                                   coords_x_last coords_y_last  \
season   game_id    game_period                                  
20202021 2020020163 1           12          96.0          -1.0   
                                16          69.0          22.0   
                                17         -39.0         -15.0   
                                28         -81.0         -37.0   
                                30         -93.0          29.0   

                                    same_team_last  is_home  score_state  \
season   game_id    game_period                                            
20202021 2020020163 1           12               0        1            0   
                                16               1        0            0   
                                17               1  

In [ ]:
df.tail(10)

In [ ]:
df.seconds_since_last.head()

In [ ]:
df.head()

In [13]:
df.columns

Index(['season', 'game_id', 'game_date', 'session', 'event_index',
       'game_period', 'game_seconds', 'clock_time', 'event_type',
       'event_description', 'event_detail', 'event_zone', 'event_team',
       'event_player_1', 'event_player_2', 'event_player_3', 'event_length',
       'coords_x', 'coords_y', 'num_on', 'num_off', 'players_on',
       'players_off', 'home_on_1', 'home_on_2', 'home_on_3', 'home_on_4',
       'home_on_5', 'home_on_6', 'home_on_7', 'away_on_1', 'away_on_2',
       'away_on_3', 'away_on_4', 'away_on_5', 'away_on_6', 'away_on_7',
       'home_goalie', 'away_goalie', 'home_team', 'away_team', 'home_skaters',
       'away_skaters', 'home_score', 'away_score', 'game_score_state',
       'game_strength_state', 'home_zone', 'pbp_distance', 'event_distance',
       'event_angle', 'home_zonestart', 'face_index', 'pen_index',
       'shift_index', 'pred_goal', 'pre_event_zone', 'pre_event_zone_2',
       'pre_event_zone_3', 'event_distance_long', 'event_distance_l

0